In [179]:
!pip3 install dash==2.0.0

!pip install yfinance --upgrade --no-cache-dir

import pandas as pd
import dash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from glob import glob
from time import strftime, sleep
import numpy as np
from datetime import datetime
from pandas_datareader import data as pdr
from pandas.tseries.offsets import BDay
import yfinance as yf
yf.pdr_override()




  Using cached dash-2.0.0-py3-none-any.whl (7.3 MB)
  Using cached dash_table-5.0.0-py3-none-any.whl
  Using cached dash_html_components-2.0.0-py3-none-any.whl
  Using cached dash_core_components-2.0.0-py3-none-any.whl
  Attempting uninstall: dash-table
    Found existing installation: dash-table 4.10.1
    Uninstalling dash-table-4.10.1:
      Successfully uninstalled dash-table-4.10.1
  Attempting uninstall: dash-html-components
    Found existing installation: dash-html-components 1.1.1
    Uninstalling dash-html-components-1.1.1:
      Successfully uninstalled dash-html-components-1.1.1
  Attempting uninstall: dash-core-components
    Found existing installation: dash-core-components 1.12.1
    Uninstalling dash-core-components-1.12.1:
      Successfully uninstalled dash-core-components-1.12.1
  Attempting uninstall: dash
    Found existing installation: dash 1.16.2
    Uninstalling dash-1.16.2:
      Successfully uninstalled dash-1.16.2


In [134]:
# simple function to make headers nicer
def clean_header(df):
    df.columns = df.columns.str.strip().str.lower().str.replace('.', '').str.replace('(', '').str.replace(')', '').str.replace(' ', '_').str.replace('_/_', '/')

# timestamp for file names
def get_now():
    now = datetime.now().strftime('%Y-%m-%d_%Hh%Mm')
    return now

In [135]:
last_file = glob('/Users/rohit/Downloads/transactions_finaldf_2021-05-15_12h19m_repo.xlsx')[-1] # path to file in the folder
print(last_file[-(len(last_file))+(last_file.rfind('/')+1):])
all_transactions = pd.read_excel(last_file)
all_transactions.date = pd.to_datetime(all_transactions.date, format='%d/%m/%Y')

transactions_finaldf_2021-05-15_12h19m_repo.xlsx


In [136]:
all_tickers = list(all_transactions['ticker'].unique())
# some tickers may have been delisted. need to blacklist them here
blacklist = ['VSLR', 'HTZ']
filt_tickers = [tick for tick in all_tickers if tick not in blacklist]
print('You traded {} different stocks'.format(len(all_tickers)))

You traded 41 different stocks


In [137]:
# all transactions without the delisted stocks
final_filtered = all_transactions[~all_transactions.ticker.isin(blacklist)]




In [138]:
#Collecting the price history for all tickers

In [139]:
ly = datetime.today().year-1
today = datetime.today()
start_sp = datetime(2019, 1, 1)
end_sp = today
start_stocks = datetime(2019, 1, 1)
end_stocks = today
start_ytd = datetime(ly, 12, 31) + BDay(1)

def get(tickers, startdate, enddate):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['ticker', 'date']))
               
all_data = get(filt_tickers, start_stocks, end_stocks)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [140]:
all_data 

Open        High         Low       Close   Adj Close  \
ticker date                                                                     
MMM    2018-12-31  190.339996  191.649994  188.500000  190.539993  172.244705   
       2019-01-02  187.820007  190.990005  186.699997  190.949997  172.615341   
       2019-01-03  188.279999  188.279999  182.889999  183.759995  166.115723   
       2019-01-04  186.750000  191.979996  186.029999  191.320007  172.949829   
       2019-01-07  191.360001  192.300003  188.660004  190.880005  172.552048   
...                       ...         ...         ...         ...         ...   
GILT   2022-02-01    7.550000    7.650000    7.450000    7.590000    7.590000   
       2022-02-02    7.730000    7.880000    7.660000    7.790000    7.790000   
       2022-02-03    7.620000    7.830000    7.600000    7.750000    7.750000   
       2022-02-04    7.690000    7.750000    7.500000    7.550000    7.550000   
       2022-02-07    7.710000    7.740000    7.550000    7.620000    7.620000   

                    Volume  
ticker date                 
MMM    2018-12-31  1804400  
       2019-01-02  2475200  
       2019-01-03  3358200  
       2019-01-04  2995100  
       2019-01-07  2162200  
...                    ...  
GILT   2022-02-01   232700  
       2022-02-02   364900  
       2022-02-03   216600  
       2022-02-04   194900  
       2022-02-07   179200  

[29580 rows x 6 columns]

In [141]:
clean_header(all_data)

/var/folders/0p/kfv9p6wd4l91mbwp20nhpg_40000gn/T/ipykernel_1410/2704269921.py:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/var/folders/0p/kfv9p6wd4l91mbwp20nhpg_40000gn/T/ipykernel_1410/2704269921.py:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/var/folders/0p/kfv9p6wd4l91mbwp20nhpg_40000gn/T/ipykernel_1410/2704269921.py:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [142]:
# saving all stock prices individually to the specified folder
for tick in filt_tickers:
    all_data.loc[tick].to_csv('/Users/rohit/Downloads/outputs/price_hist/{}_price_hist.csv'.format(tick))

In [143]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 29580 entries, ('MMM', Timestamp('2018-12-31 00:00:00')) to ('GILT', Timestamp('2022-02-07 00:00:00'))
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   open       29580 non-null  float64
 1   high       29580 non-null  float64
 2   low        29580 non-null  float64
 3   close      29580 non-null  float64
 4   adj_close  29580 non-null  float64
 5   volume     29580 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 1.5+ MB


In [144]:
MEGA_DICT = {}  # you have to create it first
min_date = '2020-01-01'  # optional
TX_COLUMNS = ['date','ticker', 'cashflow', 'cml_units', 'cml_cost', 'gain_loss']
tx_filt = all_transactions[TX_COLUMNS]  # keeping just the most relevant ones for now

for ticker in filt_tickers:
    prices_df = all_data[all_data.index.get_level_values('ticker').isin([ticker])].reset_index()
    ## Can add more columns like volume!
    PX_COLS = ['date', 'adj_close']
    prices_df = prices_df[prices_df.date >= min_date][PX_COLS].set_index(['date'])
    # Making sure we get sameday transactions
    tx_df = tx_filt[tx_filt.ticker==ticker].groupby('date').agg({'cashflow': 'sum',
                                                                 'cml_units': 'last',
                                                                 'cml_cost': 'last',
                                                                 'gain_loss': 'sum'})
    # Merging price history and transactions dataframe
    tx_and_prices = pd.merge(prices_df, tx_df, how='outer', left_index=True, right_index=True).fillna(0)
    # This is to fill the days that were not in our transaction dataframe
    tx_and_prices['cml_units'] = tx_and_prices['cml_units'].replace(to_replace=0, method='ffill')
    tx_and_prices['cml_cost'] = tx_and_prices['cml_cost'].replace(to_replace=0, method='ffill')
    tx_and_prices['gain_loss'] = tx_and_prices['gain_loss'].replace(to_replace=0, method='ffill')
    # Cumulative sum for the cashflow
    tx_and_prices['cashflow'] = tx_and_prices['cashflow'].cumsum()
    tx_and_prices['avg_price'] = (tx_and_prices['cml_cost']/tx_and_prices['cml_units'])
    tx_and_prices['mktvalue'] = (tx_and_prices['cml_units']*tx_and_prices['adj_close'])
    tx_and_prices = tx_and_prices.add_prefix(ticker+'_')
    # Once we're happy with the dataframe, add it to the dictionary
    MEGA_DICT[ticker] = tx_and_prices.round(3)

In [145]:
MEGA_DICT['RUN'].tail()

,RUN_adj_close,RUN_cashflow,RUN_cml_units,RUN_cml_cost,RUN_gain_loss,RUN_avg_price,RUN_mktvalue
date,,,,,,,
2022-02-01,26.01,201.08,10.0,336.386,429.891,33.639,260.1
2022-02-02,24.78,201.08,10.0,336.386,429.891,33.639,247.8
2022-02-03,23.04,201.08,10.0,336.386,429.891,33.639,230.4
2022-02-04,25.11,201.08,10.0,336.386,429.891,33.639,251.1
2022-02-07,23.89,201.08,10.0,336.386,429.891,33.639,238.9


In [146]:
MEGA_DF = pd.concat(MEGA_DICT.values(), axis=1)
MEGA_DF.to_csv('/Users/rohit/Downloads/outputs/mega/MEGA_DF_{}.csv'.format(get_now()))  # optional
MEGA_DF.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 530 entries, 2020-01-02 to 2022-02-07
Columns: 273 entries, MMM_adj_close to GILT_mktvalue
dtypes: float64(273)
memory usage: 1.1 MB


In [147]:
MEGA_DF

,MMM_adj_close,MMM_cashflow,MMM_cml_units,MMM_cml_cost,MMM_gain_loss,MMM_avg_price,MMM_mktvalue,TSLA_adj_close,TSLA_cashflow,TSLA_cml_units,...,TLRY_gain_loss,TLRY_avg_price,TLRY_mktvalue,GILT_adj_close,GILT_cashflow,GILT_cml_units,GILT_cml_cost,GILT_gain_loss,GILT_avg_price,GILT_mktvalue
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,168.199,0.00,0.0,0.00,0.0,NaN,0.000,86.052,0.00,0.0,...,0.00,NaN,0.000,6.922,0.00,0.0,0.000,0.000,NaN,0.000
2020-01-03,166.751,0.00,0.0,0.00,0.0,NaN,0.000,88.602,0.00,0.0,...,0.00,NaN,0.000,6.905,0.00,0.0,0.000,0.000,NaN,0.000
2020-01-06,166.910,0.00,0.0,0.00,0.0,NaN,0.000,90.308,0.00,0.0,...,0.00,NaN,0.000,7.112,0.00,0.0,0.000,0.000,NaN,0.000
2020-01-07,166.237,0.00,0.0,0.00,0.0,NaN,0.000,93.812,0.00,0.0,...,0.00,NaN,0.000,7.051,0.00,0.0,0.000,0.000,NaN,0.000
2020-01-08,168.788,-180.95,1.0,180.95,0.0,180.95,168.788,98.428,-477.00,5.0,...,0.00,NaN,0.000,7.155,0.00,0.0,0.000,0.000,NaN,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-01,167.430,-180.95,1.0,180.95,0.0,180.95,167.430,931.250,66.29,1.5,...,-25.37,62.141,5.461,7.590,-269.62,15.9,246.417,-23.203,15.498,120.681
2022-02-02,166.660,-180.95,1.0,180.95,0.0,180.95,166.660,905.660,66.29,1.5,...,-25.37,62.141,5.250,7.790,-269.62,15.9,246.417,-23.203,15.498,123.861
2022-02-03,164.390,-180.95,1.0,180.95,0.0,180.95,164.390,891.140,66.29,1.5,...,-25.37,62.141,4.961,7.750,-269.62,15.9,246.417,-23.203,15.498,123.225


In [148]:
MEGA_DF = pd.concat(MEGA_DICT.values(), axis=1)
MEGA_DF.to_csv('/Users/rohit/Downloads/outputs/mega/MEGA_DF_{}.csv'.format(get_now()))  # optional
MEGA_DF.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 530 entries, 2020-01-02 to 2022-02-07
Columns: 273 entries, MMM_adj_close to GILT_mktvalue
dtypes: float64(273)
memory usage: 1.1 MB


In [149]:
last_file = glob('/Users/rohit/Downloads/outputs/mega/MEGA*.csv')[-1] # path to file in the folder
print(last_file[-(len(last_file))+(last_file.rfind('/')+1):])
MEGA_DF = pd.read_csv(last_file)

MEGA_DF['date'] = pd.to_datetime(MEGA_DF['date'])
MEGA_DF.set_index('date', inplace=True)

MEGA_DF_2022-02-08_17h57m.csv


In [150]:
#Portfolio DF

In [151]:
portf_allvalues = MEGA_DF.filter(regex='mktvalue').fillna(0) #  getting just the market value of each ticker
portf_allvalues['portf_value'] = portf_allvalues.sum(axis=1) # summing all market values
portf_allvalues['portf_value']

date
2020-01-02        0.000
2020-01-03        0.000
2020-01-06        0.000
2020-01-07        0.000
2020-01-08      660.928
                ...    
2022-02-01    20726.005
2022-02-02    20454.445
2022-02-03    19712.727
2022-02-04    20290.588
2022-02-07    20318.376
Name: portf_value, Length: 530, dtype: float64

In [152]:
# For the S&P500 price return
sp500 = pdr.get_data_yahoo('^GSPC', start_stocks, end_sp)
clean_header(sp500)

[*********************100%***********************]  1 of 1 completed


/var/folders/0p/kfv9p6wd4l91mbwp20nhpg_40000gn/T/ipykernel_1410/2704269921.py:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/var/folders/0p/kfv9p6wd4l91mbwp20nhpg_40000gn/T/ipykernel_1410/2704269921.py:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/var/folders/0p/kfv9p6wd4l91mbwp20nhpg_40000gn/T/ipykernel_1410/2704269921.py:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [154]:
#getting the pct change
portf_allvalues = portf_allvalues.join(sp500['adj_close'], how='inner')
portf_allvalues.rename(columns={'adj_close': 'sp500_mktvalue'}, inplace=True)
portf_allvalues['ptf_value_pctch'] = (portf_allvalues['portf_value'].pct_change()*100).round(2)
portf_allvalues['sp500_pctch'] = (portf_allvalues['sp500_mktvalue'].pct_change()*100).round(2)
portf_allvalues['ptf_value_diff'] = (portf_allvalues['portf_value'].diff()).round(2)
portf_allvalues['sp500_diff'] = (portf_allvalues['sp500_mktvalue'].diff()).round(2)

In [155]:
portf_allvalues.head()

,MMM_mktvalue,TSLA_mktvalue,AAPL_mktvalue,RUN_mktvalue,ZM_mktvalue,DIS_mktvalue,IBM_mktvalue,AMD_mktvalue,FB_mktvalue,BA_mktvalue,...,PLTR_mktvalue,PLL_mktvalue,TLRY_mktvalue,GILT_mktvalue,portf_value,sp500_mktvalue,ptf_value_pctch,sp500_pctch,ptf_value_diff,sp500_diff
2020-01-02,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,3257.850098,NaN,NaN,NaN,NaN
2020-01-03,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,3234.850098,NaN,-0.71,0.00,-23.00
2020-01-06,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,3246.280029,NaN,0.35,0.00,11.43
2020-01-07,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,3237.179932,NaN,-0.28,0.00,-9.10
2020-01-08,168.788,492.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,660.928,3253.050049,inf,0.49,660.93,15.87


In [156]:
# KPI's for portfolio
kpi_portfolio7d_abs = portf_allvalues.tail(7).ptf_value_diff.sum().round(2)
kpi_portfolio15d_abs = portf_allvalues.tail(15).ptf_value_diff.sum().round(2)
kpi_portfolio30d_abs = portf_allvalues.tail(30).ptf_value_diff.sum().round(2)
kpi_portfolio200d_abs = portf_allvalues.tail(200).ptf_value_diff.sum().round(2)
kpi_portfolio7d_pct = (kpi_portfolio7d_abs/portf_allvalues.tail(7).portf_value[0]).round(3)*100
kpi_portfolio15d_pct = (kpi_portfolio15d_abs/portf_allvalues.tail(15).portf_value[0]).round(3)*100
kpi_portfolio30d_pct = (kpi_portfolio30d_abs/portf_allvalues.tail(30).portf_value[0]).round(3)*100
kpi_portfolio200d_pct = (kpi_portfolio200d_abs/portf_allvalues.tail(200).portf_value[0]).round(3)*100

In [157]:
# KPI's for S&P500
kpi_sp500_7d_abs = portf_allvalues.tail(7).sp500_diff.sum().round(2)
kpi_sp500_15d_abs = portf_allvalues.tail(15).sp500_diff.sum().round(2)
kpi_sp500_30d_abs = portf_allvalues.tail(30).sp500_diff.sum().round(2)
kpi_sp500_200d_abs = portf_allvalues.tail(200).sp500_diff.sum().round(2)
kpi_sp500_7d_pct = (kpi_sp500_7d_abs/portf_allvalues.tail(7).sp500_mktvalue[0]).round(3)*100
kpi_sp500_15d_pct = (kpi_sp500_15d_abs/portf_allvalues.tail(15).sp500_mktvalue[0]).round(3)*100
kpi_sp500_30d_pct = (kpi_sp500_30d_abs/portf_allvalues.tail(30).sp500_mktvalue[0]).round(3)*100
kpi_sp500_200d_pct = (kpi_sp500_200d_abs/portf_allvalues.tail(200).sp500_mktvalue[0]).round(3)*100

In [158]:

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import plotly.express as px
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import dash_table
from jupyter_dash import JupyterDash


In [159]:
initial_date = '2020-05-30'  # do not use anything earlier than your first trade
plotlydf_portfval = portf_allvalues[portf_allvalues.index > initial_date]
plotlydf_portfval = plotlydf_portfval[['portf_value', 'sp500_mktvalue', 'ptf_value_pctch',
                                     'sp500_pctch', 'ptf_value_diff', 'sp500_diff']].reset_index().round(2)
# calculating cumulative growth since initial date
plotlydf_portfval['ptf_growth'] = plotlydf_portfval.portf_value/plotlydf_portfval['portf_value'].iloc[0]
plotlydf_portfval['sp500_growth'] = plotlydf_portfval.sp500_mktvalue/plotlydf_portfval['sp500_mktvalue'].iloc[0]
plotlydf_portfval.rename(columns={'index': 'date'}, inplace=True)  # needed for later
plotlydf_portfval

,date,portf_value,sp500_mktvalue,ptf_value_pctch,sp500_pctch,ptf_value_diff,sp500_diff,ptf_growth,sp500_growth
0,2020-06-01,10872.44,3055.73,8.42,0.38,844.61,11.42,1.000000,1.000000
1,2020-06-02,11008.77,3080.82,1.25,0.82,136.33,25.09,1.012539,1.008211
2,2020-06-03,11518.20,3122.87,4.63,1.36,509.43,42.05,1.059394,1.021972
3,2020-06-04,11311.45,3112.35,-1.79,-0.34,-206.75,-10.52,1.040378,1.018529
4,2020-06-05,11843.97,3193.93,4.71,2.62,532.52,81.58,1.089357,1.045227
...,...,...,...,...,...,...,...,...,...
422,2022-02-01,20726.00,4546.54,1.46,0.69,297.94,30.99,1.906288,1.487874
423,2022-02-02,20454.44,4589.38,-1.31,0.94,-271.56,42.84,1.881311,1.501893
424,2022-02-03,19712.73,4477.44,-3.63,-2.44,-741.72,-111.94,1.813092,1.465260
425,2022-02-04,20290.59,4500.53,2.93,0.52,577.86,23.09,1.866241,1.472817


In [160]:
CHART_THEME = 'plotly_dark'  # others include seaborn, ggplot2, plotly_dark

chart_ptfvalue = go.Figure()  # generating a figure that will be updated in the following lines
chart_ptfvalue.add_trace(go.Scatter(x=plotlydf_portfval.date, y=plotlydf_portfval.portf_value,
                    mode='lines',  # you can also use "lines+markers", or just "markers"
                    name='Global Value'))
chart_ptfvalue.layout.template = CHART_THEME
chart_ptfvalue.layout.height=500
chart_ptfvalue.update_layout(margin = dict(t=50, b=50, l=25, r=25))  # this will help you optimize the chart space
chart_ptfvalue.update_layout(
#     title='Global Portfolio Value (USD $)',
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='Value: $ USD',
        titlefont_size=14,
        tickfont_size=12,
        ))
# chart_ptfvalue.update_xaxes(rangeslider_visible=False)
# chart_ptfvalue.update_layout(showlegend=False)
chart_ptfvalue.show()

In [161]:
import plotly.io as pio
list(pio.templates)  # doctest: +ELLIPSIS

['ggplot2',
 'seaborn',
 'simple_white',
 'plotly',
 'plotly_white',
 'plotly_dark',
 'presentation',
 'xgridoff',
 'ygridoff',
 'gridon',
 'none']

In [162]:
plotlydf_portfval

,date,portf_value,sp500_mktvalue,ptf_value_pctch,sp500_pctch,ptf_value_diff,sp500_diff,ptf_growth,sp500_growth
0,2020-06-01,10872.44,3055.73,8.42,0.38,844.61,11.42,1.000000,1.000000
1,2020-06-02,11008.77,3080.82,1.25,0.82,136.33,25.09,1.012539,1.008211
2,2020-06-03,11518.20,3122.87,4.63,1.36,509.43,42.05,1.059394,1.021972
3,2020-06-04,11311.45,3112.35,-1.79,-0.34,-206.75,-10.52,1.040378,1.018529
4,2020-06-05,11843.97,3193.93,4.71,2.62,532.52,81.58,1.089357,1.045227
...,...,...,...,...,...,...,...,...,...
422,2022-02-01,20726.00,4546.54,1.46,0.69,297.94,30.99,1.906288,1.487874
423,2022-02-02,20454.44,4589.38,-1.31,0.94,-271.56,42.84,1.881311,1.501893
424,2022-02-03,19712.73,4477.44,-3.63,-2.44,-741.72,-111.94,1.813092,1.465260
425,2022-02-04,20290.59,4500.53,2.93,0.52,577.86,23.09,1.866241,1.472817


In [163]:
fig2 = go.Figure(data=[
    go.Bar(name='Portfolio', x=plotlydf_portfval['date'], y=plotlydf_portfval['ptf_value_pctch']),
    go.Bar(name='SP500', x=plotlydf_portfval['date'], y=plotlydf_portfval['sp500_pctch'])
])
# Change the bar mode
fig2.update_layout(barmode='group')
fig2.layout.template = CHART_THEME
fig2.layout.height=300
fig2.update_layout(margin = dict(t=50, b=50, l=25, r=25))
fig2.update_layout(
#     title='% variation - Portfolio vs SP500',
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='% change',
        titlefont_size=14,
        tickfont_size=12,
        ))
fig2.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99))

fig2.show()

In [164]:
df = plotlydf_portfval[['date', 'ptf_growth', 'sp500_growth']].copy().round(3)
df['month'] = df.date.dt.month_name()  # date column should be formatted as datetime
df['weekday'] = df.date.dt.day_name()  # could be interesting to analyze weekday returns later
df['year'] = df.date.dt.year
df['weeknumber'] = df.date.dt.week    # could be interesting to try instead of timeperiod
df['timeperiod'] = df.year.astype(str) + ' - ' + df.date.dt.month.astype(str).str.zfill(2)
df.head(5)

/var/folders/0p/kfv9p6wd4l91mbwp20nhpg_40000gn/T/ipykernel_1410/3218988876.py:5: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.



,date,ptf_growth,sp500_growth,month,weekday,year,weeknumber,timeperiod
0,2020-06-01,1.000,1.000,June,Monday,2020,23,2020 - 06
1,2020-06-02,1.013,1.008,June,Tuesday,2020,23,2020 - 06
2,2020-06-03,1.059,1.022,June,Wednesday,2020,23,2020 - 06
3,2020-06-04,1.040,1.019,June,Thursday,2020,23,2020 - 06
4,2020-06-05,1.089,1.045,June,Friday,2020,23,2020 - 06


In [165]:
# getting the percentage change for each period. the first period will be NaN
sp = df.reset_index().groupby('timeperiod').last()['sp500_growth'].pct_change()*100
ptf = df.reset_index().groupby('timeperiod').last()['ptf_growth'].pct_change()*100
plotlydf_growth_compare = pd.merge(ptf, sp, on='timeperiod').reset_index().round(3)
plotlydf_growth_compare.head()

,timeperiod,ptf_growth,sp500_growth
0,2020 - 06,NaN,NaN
1,2020 - 07,9.550,5.419
2,2020 - 08,33.097,7.009
3,2020 - 09,9.851,-3.843
4,2020 - 10,-5.284,-2.816


In [166]:
fig_growth2 = go.Figure()
fig_growth2.layout.template = CHART_THEME
fig_growth2.add_trace(go.Bar(
    x=plotlydf_growth_compare.timeperiod,
    y=plotlydf_growth_compare.ptf_growth.round(2),
    name='Portfolio'
))
fig_growth2.add_trace(go.Bar(
    x=plotlydf_growth_compare.timeperiod,
    y=plotlydf_growth_compare.sp500_growth.round(2),
    name='S&P 500',
))
fig_growth2.update_layout(barmode='group')
fig_growth2.layout.height=300
fig_growth2.update_layout(margin = dict(t=50, b=50, l=25, r=25))
fig_growth2.update_layout(
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='% change',
        titlefont_size=13,
        tickfont_size=12,
        ))

fig_growth2.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99))
fig_growth2.show()

In [167]:
indicators_ptf = go.Figure()
indicators_ptf.layout.template = CHART_THEME
indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio7d_pct,
    number = {'suffix': " %"},
    title = {"text": "<br><span style='font-size:0.7em;color:gray'>7 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_7d_pct, 'relative': False},
    domain = {'row': 0, 'column': 0}))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio15d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>15 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_15d_pct, 'relative': False},
    domain = {'row': 1, 'column': 0}))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio30d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>30 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_30d_pct, 'relative': False},
    domain = {'row': 2, 'column': 0}))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio200d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>200 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_200d_pct, 'relative': False},
    domain = {'row': 3, 'column': 1}))

indicators_ptf.update_layout(
    grid = {'rows': 4, 'columns': 1, 'pattern': "independent"},
    margin=dict(l=50, r=50, t=30, b=30)
)


In [168]:
indicators_sp500 = go.Figure()
indicators_sp500.layout.template = CHART_THEME
indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_7d_pct,
    number = {'suffix': " %"},
    title = {"text": "<br><span style='font-size:0.7em;color:gray'>7 Days</span>"},
    domain = {'row': 0, 'column': 0}))

indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_15d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>15 Days</span>"},
    domain = {'row': 1, 'column': 0}))

indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_30d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>30 Days</span>"},
    domain = {'row': 2, 'column': 0}))

indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_200d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>200 Days</span>"},
    domain = {'row': 3, 'column': 1}))

indicators_sp500.update_layout(
    grid = {'rows': 4, 'columns': 1, 'pattern': "independent"},
    margin=dict(l=50, r=50, t=30, b=30)
)

In [169]:
# Getting the prices for the TOP Stocks

In [170]:
last_positions = final_filtered.groupby(['ticker']).agg({'cml_units': 'last', 'cml_cost': 'last',
                                                'gain_loss': 'sum', 'cashflow': 'sum'}).reset_index()

In [173]:
%%time
curr_prices = []
for tick in last_positions['ticker']:
    stonk = yf.Ticker(tick)
    price = stonk.info['regularMarketPrice']
    curr_prices.append(price)
    print(f'Done for {tick}')
len(curr_prices)

Done for AAPL
Done for AMD
Done for AMZN
Done for BA
Done for CVX
Done for DIS
Done for DKNG
Done for FB
Done for FVRR
Done for GILT
Done for HD
Done for IBM
Done for LULU
Done for LUNA
Done for MCD
Done for MGM
Done for MMM
Done for MO
Done for MRNA
Done for NIO
Done for NVDA
Done for NWS
Done for PLL
Done for PLTR
Done for PLUG
Done for PTON
Done for QCOM
Done for RUN
Done for SHOP
Done for SYF
Done for TLRY
Done for TSLA
Done for TTD
Done for TWTR
Done for VIAC
Done for WMT
Done for WYNN
Done for XLNX
Done for ZM
CPU times: user 12.9 s, sys: 1.09 s, total: 14 s
Wall time: 4min 7s


39

In [174]:
last_positions['price'] = curr_prices
last_positions['current_value'] = (last_positions.price * last_positions.cml_units).round(2)
last_positions['avg_price'] = (last_positions.cml_cost / last_positions.cml_units).round(2)
last_positions = last_positions.sort_values(by='current_value', ascending=False)

In [175]:
last_positions

,ticker,cml_units,cml_cost,gain_loss,cashflow,price,current_value,avg_price
0,AAPL,15.0000,1139.9500,0.0000,-1139.95,171.66,2574.90,76.00
2,AMZN,0.6800,1687.6300,0.0000,-1687.63,3158.71,2147.92,2481.81
3,BA,7.4000,1279.6100,0.0000,-1279.61,211.92,1568.21,172.92
31,TSLA,1.5000,387.1660,453.4560,66.29,907.34,1361.01,258.11
8,FVRR,14.2500,867.8250,183.2550,-684.57,81.01,1154.39,60.90
34,VIAC,25.5000,400.6050,218.7850,-181.82,34.00,867.00,15.71
4,CVX,5.4000,482.2000,0.0000,-482.20,138.55,748.17,89.30
11,IBM,5.0000,621.4000,0.0000,-621.40,137.24,686.20,124.28
22,PLL,12.8000,472.7700,0.0000,-472.77,47.64,609.79,36.94
18,MRNA,3.7500,251.8500,0.0000,-251.85,160.27,601.01,67.16


In [176]:
donut_top = go.Figure()
donut_top.layout.template = CHART_THEME
donut_top.add_trace(go.Pie(labels=last_positions.head(15).ticker, values=last_positions.head(15).current_value))
donut_top.update_traces(hole=.7, hoverinfo="label+value+percent")
donut_top.update_traces(textposition='outside', textinfo='label+value')
donut_top.update_layout(showlegend=False)
donut_top.update_layout(margin = dict(t=50, b=50, l=25, r=25))
donut_top.show()

In [181]:
SIDEBAR_STYLE = {
    'position': 'fixed',
    'top': 0,
    'left': 0,
    'bottom': 0,
    'width': '12rem',
    'padding': '2rem 1rem',
    'background-color': 'lightgray',
}
CONTENT_STYLE = {
    'margin-left': '15rem',
    'margin-right': '2rem',
    'padding': '2rem' '1rem',
}

child = dbc.Container(
    [
        dbc.Row(dbc.Col(html.H2('PORTFOLIO OVERVIEW', className='text-center text-primary, mb-3'))),
        dbc.Row([
            dbc.Col([
            html.H5('Total Portfolio Value ($USD)', className='text-center'),
            dcc.Graph(id='chrt-portfolio-main',
                      figure=chart_ptfvalue,
                      style={'height':550}),
            html.Hr(),

            ],
                width={'size': 8, 'offset': 0, 'order': 1}),
            dbc.Col([
            html.H5('Portfolio', className='text-center'),
            dcc.Graph(id='indicators-ptf',
                      figure=indicators_ptf,
                      style={'height':550}),
            html.Hr()
            ],
                width={'size': 2, 'offset': 0, 'order': 2}),
            dbc.Col([
            html.H5('S&P500', className='text-center'),
            dcc.Graph(id='indicators-sp',
                      figure=indicators_sp500,
                      style={'height':550}),
            html.Hr()
            ],
                width={'size': 2, 'offset': 0, 'order': 3}),
        ]),  # end of second row
        dbc.Row([
            dbc.Col([
                html.H5('Monthly Return (%)', className='text-center'),
                dcc.Graph(id='chrt-portfolio-secondary',
                      figure=fig_growth2,
                      style={'height':380}),
            ],
                width={'size': 8, 'offset': 0, 'order': 1}),
            dbc.Col([
                html.H5('Top 15 Holdings', className='text-center'),
                dcc.Graph(id='pie-top15',
                      figure = donut_top,
                      style={'height':380}),
            ],
                width={'size': 4, 'offset': 0, 'order': 2}),
        ])
        
    ], fluid=True)

sidebar = html.Div(
    [
#         html.H5("Navigation Menu", className='display-6'),
        html.Hr(),
        html.P('Navigation Menu', className='text-center'),
        
        dbc.Nav(
            [
                dbc.NavLink('Home', href="/", active='exact'),
                dbc.NavLink('Page2', href="/page-2", active='exact')
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id='page-content', children=child, style=CONTENT_STYLE)

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SPACELAB])
# app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Location(id='url'),
    sidebar,
    content
])

if __name__ == "__main__":
    app.run_server(debug=True, port=8056)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8056): Max retries exceeded with url: /_alive_12d3b25b-21ab-4fb1-bede-ca287751b400 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fbf2174c760>: Failed to establish a new connection: [Errno 61] Connection refused'))

Exception in thread Thread-99:
Traceback (most recent call last):
  File "/Users/rohit/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/rohit/opt/anaconda3/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rohit/opt/anaconda3/lib/python3.9/site-packages/retrying.py", line 49, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "/Users/rohit/opt/anaconda3/lib/python3.9/site-packages/retrying.py", line 212, in call
    raise attempt.get()
  File "/Users/rohit/opt/anaconda3/lib/python3.9/site-packages/retrying.py", line 247, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "/Users/rohit/opt/anaconda3/lib/python3.9/site-packages/six.py", line 719, in reraise
    raise value
  File "/Users/rohit/opt/anaconda3/lib/python3.9/site-packages/retrying.py", line 200, in call
    attempt = Attempt(fn(*args, **kwargs), attempt_number, False)

In [ ]:
app._terminate_server_for_port("localhost",8058)

In [180]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = dbc.Container(
    [
        dbc.Row(dbc.Col(html.H2('PORTFOLIO OVERVIEW', className='text-center text-primary, mb-3'))),  # header row
        
        dbc.Row([  # start of second row
            dbc.Col([  # first column on second row
            html.H5('Total Portfolio Value ($USD)', className='text-center'),
            dcc.Graph(id='chrt-portfolio-main',
                      figure=chart_ptfvalue,
                      style={'height':550}),
            html.Hr(),
            ], width={'size': 8, 'offset': 0, 'order': 1}),  # width first column on second row
            dbc.Col([  # second column on second row
            html.H5('Portfolio', className='text-center'),
            dcc.Graph(id='indicators-ptf',
                      figure=indicators_ptf,
                      style={'height':550}),
            html.Hr()
            ], width={'size': 2, 'offset': 0, 'order': 2}),  # width second column on second row
            dbc.Col([  # third column on second row
            html.H5('S&P500', className='text-center'),
            dcc.Graph(id='indicators-sp',
                      figure=indicators_sp500,
                      style={'height':550}),
            html.Hr()
            ], width={'size': 2, 'offset': 0, 'order': 3}),  # width third column on second row
        ]),  # end of second row
        
        dbc.Row([  # start of third row
            dbc.Col([  # first column on third row
                html.H5('Monthly Return (%)', className='text-center'),
                dcc.Graph(id='chrt-portfolio-secondary',
                      figure=fig_growth2,
                      style={'height':380}),
            ], width={'size': 8, 'offset': 0, 'order': 1}),  # width first column on second row
            dbc.Col([  # second column on third row
                html.H5('Top 15 Holdings', className='text-center'),
                dcc.Graph(id='pie-top15',
                      figure = donut_top,
                      style={'height':380}),
            ], width={'size': 4, 'offset': 0, 'order': 2}),  # width second column on second row
        ])  # end of third row
        
    ], fluid=True)

if __name__ == "__main__":
    app.run_server(debug=True, port=8058)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8058): Max retries exceeded with url: /_alive_12d3b25b-21ab-4fb1-bede-ca287751b400 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fbf010014c0>: Failed to establish a new connection: [Errno 61] Connection refused'))

Exception in thread Thread-98:
Traceback (most recent call last):
  File "/Users/rohit/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/rohit/opt/anaconda3/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rohit/opt/anaconda3/lib/python3.9/site-packages/retrying.py", line 49, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "/Users/rohit/opt/anaconda3/lib/python3.9/site-packages/retrying.py", line 212, in call
    raise attempt.get()
  File "/Users/rohit/opt/anaconda3/lib/python3.9/site-packages/retrying.py", line 247, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "/Users/rohit/opt/anaconda3/lib/python3.9/site-packages/six.py", line 719, in reraise
    raise value
  File "/Users/rohit/opt/anaconda3/lib/python3.9/site-packages/retrying.py", line 200, in call
    attempt = Attempt(fn(*args, **kwargs), attempt_number, False)

In [ ]:
# Getting sector and Industry

In [ ]:
 %%time
tickers_string = ','.join(map(str, filt_tickers))
screener_overview = 'https://finviz.com/screener.ashx?v=111&t=' + tickers_string + '&r='
p = len(filt_tickers)//20+1
finviz_merged_raw = biz_extractor.get_finviz_screener(screener_overview, p)

In [ ]:
cols_clean = ['Ticker', 'Company', 'Sector', 'Industry', 'Price', 'P/E']
finviz_merged_clean = finviz_merged_raw[cols_clean]
clean_header(finviz_merged_clean)
finviz_merged_clean.info()

In [ ]:
finviz_merged_clean.loc[:, 'price'] = finviz_merged_clean.copy()['price'].astype(float)

In [ ]:
final_lastpositions = pd.merge(finviz_merged_clean, last_positions, left_on='ticker', right_on='ticker')
final_lastpositions['current_value'] = final_lastpositions.price * final_lastpositions.cml_units
final_lastpositions['avg_price'] = final_lastpositions.cml_cost / final_lastpositions.cml_units
final_lastpositions

In [ ]:
final_lastpositions.to_csv('/Users/rohit/Downloads/outputs/final_lastpositions.csv', index=False)

In [ ]:
final_lastpositions.info()

In [ ]:
final_lastpositions.groupby(['sector', 'industry']).agg(
    {'ticker': 'count', 'current_value': 'sum', 'cml_cost': 'sum', 'gain_loss': 'sum'}
).sort_values(by=['sector','current_value'], ascending= False).reset_index().round(2).head(15)

In [ ]:
final_lastpositions.groupby(['sector', 'industry', 'ticker']).agg(
    {'price': 'last', 'current_value': 'last', 'avg_price': 'last', 'cml_units': 'last', 'cml_cost': 'sum', 'gain_loss': 'sum'}
).sort_values(by=['sector', 'current_value'], ascending= False).reset_index().round(2)

In [ ]:
final_lastpositions.groupby(['sector', 'industry', 'ticker']).agg(
    {'price': 'last', 'current_value': 'last', 'avg_price': 'last', 'cml_units': 'last', 'cml_cost': 'sum', 'gain_loss': 'sum'}
).sort_values(by=['sector', 'current_value'], ascending= False).reset_index().round(2)